# Workshop 2: Views und CASE_WHEN
Freiwilliger Bonus: User Defined Functions (UDF)

In diesem Workshop geht es um Techniken, um häufig benutzte Queries zu automatisieren und die zu optimieren. 

* Schwierigkeit: einfach
* Prüfungsrelevant: ja (ausser UDF)
* Zeitaufwand: 30min



In [ ]:
# Hilfsfunktionen laden und ...
from util.sqlite_util import *

# ... Test-Datenbank öffnen
path = get_connection_path("chinook.sqlite")
connection = create_connection(path)

Connection to SQLite DB 2.6.0 (./data/chinook.sqlite) successful


## CASE WHEN

In [ ]:
stmt = """SELECT
	trackid, name,
	CASE
		WHEN 	milliseconds <= 60000 THEN 'short'
		WHEN 	milliseconds > 60000 AND 
			milliseconds < 300000 THEN 'medium'
		ELSE
			'long'
		END category
FROM
	tracks
LIMIT 10;
"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'For Those About To Rock (We Salute You)', 'long')
(2, 'Balls to the Wall', 'long')
(3, 'Fast As a Shark', 'medium')
(4, 'Restless and Wild', 'medium')
(5, 'Princess of the Dawn', 'long')
(6, 'Put The Finger On You', 'medium')
(7, "Let's Get It Up", 'medium')
(8, 'Inject The Venom', 'medium')
(9, 'Snowballed', 'medium')
(10, 'Evil Walks', 'medium')


## VIEW

In [ ]:
# Als erstes erstellen wir eine sogenannte View. Diese kann dann wie eine Tabelle verwendet werden.
# Views werden bei jedem Aufruf neu berechnet.
stmt="""
CREATE VIEW IF NOT EXISTS CustomerInvoices
AS
SELECT
	CustomerId,
	strftime('%Y',InvoiceDate) Year,
	SUM( total ) Total
FROM
	invoices
	GROUP BY CustomerId, strftime('%Y',InvoiceDate);
"""
cur = execute_query(connection, stmt)
print(desc_table(connection, "CustomerInvoices"))

CREATE VIEW CustomerInvoices
AS
SELECT
	CustomerId,
	strftime('%Y',InvoiceDate) Year,
	SUM( total ) Total
FROM
	invoices
	GROUP BY CustomerId, strftime('%Y',InvoiceDate)


## User Defined Functions (UDF)

In [ ]:
import sqlite3

def _toTitleCase(string):
    return str(string).title()
def _toUpperCase(string):
    return str(string).upper()

connection.create_function("title", 1, _toTitleCase)
connection.create_function("upper", 1, _toUpperCase)

stmt="""SELECT name, title(name), upper(name) FROM genres;"""

cur = execute_query(connection, stmt)
print_results(cur)

['Name', 'title(name)', 'upper(name)']
('Rock', 'Rock', 'ROCK')
('Jazz', 'Jazz', 'JAZZ')
('Metal', 'Metal', 'METAL')
('Alternative & Punk', 'Alternative & Punk', 'ALTERNATIVE & PUNK')
('Rock And Roll', 'Rock And Roll', 'ROCK AND ROLL')
('Blues', 'Blues', 'BLUES')
('Latin', 'Latin', 'LATIN')
('Reggae', 'Reggae', 'REGGAE')
('Pop', 'Pop', 'POP')
('Soundtrack', 'Soundtrack', 'SOUNDTRACK')


In [ ]:
def _formatBytes(bytes):
    fmt = "b"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "kb"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "mb"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "gb"
    if bytes > 1024:
        bytes = bytes / 1024
        fmt = "tb"
    return "%s%s" % (round(bytes,2), fmt)

connection.create_function("formatbytes", 1, _formatBytes)

stmt="""SELECT name, formatbytes(bytes) FROM tracks LIMIT 10;"""
cur = execute_query(connection, stmt)
print_results(cur)

['Name', 'formatbytes(bytes)']
('For Those About To Rock (We Salute You)', '10.65mb')
('Balls to the Wall', '5.26mb')
('Fast As a Shark', '3.81mb')
('Restless and Wild', '4.13mb')
('Princess of the Dawn', '6.0mb')
('Put The Finger On You', '6.4mb')
("Let's Get It Up", '7.28mb')
('Inject The Venom', '6.54mb')
('Snowballed', '6.29mb')
('Evil Walks', '8.21mb')


In [ ]:
import math;
connection.create_function("log", 1, math.log10)
stmt="""SELECT log(milliseconds) FROM tracks LIMIT 10;"""
cur = execute_query(connection, stmt)
print_results(cur)

['log(milliseconds)']
(5.536203539606202,)
(5.534739185524943,)
(5.3628950846379295,)
(5.401488424819678,)
(5.574515091709049,)
(5.313154054869936,)
(5.36907849469925,)
(5.323940648389206,)
(5.307714200040391,)
(5.420775674991212,)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0715ae06-549d-47db-ade6-5afadbf2cba1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>